## basic test

In [3]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [4]:
#-- REAL WORLD configuration, which is attached to the physics engine
# accessing this directly would be cheating!
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/game_scene.g")

RuntimeError: /home/ankit/robotics-course/project/robotics-course/rai/rai/Core/util.cpp:cd_file:1143(-2) couldn't change to directory '../../scenarios' from '/home/ankit/robotics-course/project/robotics-course/scenarios'

In [5]:
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

RuntimeError: /home/ankit/robotics-course/project/robotics-course/rai/rai/Kin/cameraview.cpp:addSensor:50(-2) CHECK failed: 'frame' -- frame 'camera' is not defined

In [6]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/pandasTable.g")

In [7]:
#-- using the viewer, you can view configurations or paths
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [8]:
#-- the following is the simulation loop
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    [y,J] = C.evalFeature(ry.FS.position, ["R_gripper"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ [0.,0.,-1e-1];

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

## doing things relative to an object

In [ ]:
# add a new frame to the MODEL configuration
# (Perception will later have to do exactly this: add perceived objects to the model)
obj = C.addFrame("object")

In [ ]:
# set frame parameters, associate a shape to the frame, 
obj.setPosition([.8,0,1.5])
obj.setQuaternion([1,0,.5,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])
V.setConfiguration(C)
V.recopyMeshes(C) #this is rarely necessary, only when you change meshes within C

In [ ]:
#-- the following is the simulation loop
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    [y,J] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "object"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

How could you align the gripper for a proper grasp?

In [ ]:
S=0
V=0
C=0
RealWorld=0